### Imports

In [40]:
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize , word_tokenize

<h1>Carregando Dados</h1>
<p>Dados de hospedagens cadastradas pelo ministerio do turismo</p>

In [41]:
##Carregando dados
hospedagens_trimestre01 = pd.read_csv('Datasets/MeiodeHospedagem201601TrimestreCadasturPJ.csv',encoding='latin-1',sep=';')
hospedagens_trimestre02 = pd.read_csv('Datasets/MeiodeHospedagem201602TrimestreCadasturPJ.csv',encoding='latin-1',sep=';')
hospedagens_trimestre03 = pd.read_csv('Datasets/MeiodeHospedagem201603TrimestreCadasturPJ.csv',encoding='latin-1',sep=';')
hospedagens_trimestre04 = pd.read_csv('Datasets/MeiodeHospedagem201604TrimestreCadasturPJ.csv',encoding='latin-1',sep=';')

In [42]:
## Colunas dos Dados
print(hospedagens_trimestre01.columns)
print(hospedagens_trimestre02.columns)
print(hospedagens_trimestre03.columns)
print(hospedagens_trimestre04.columns)

Index(['RAZÃO SOCIAL', 'NOME FANTASIA', 'CNPJ', 'NATUREZA JURIDICA',
       'DATA INICIO OPERAÇÃO', 'PORTE', 'SITUAÇÃO', 'TIPO ATIVIDADE',
       'SUBTIPO ATIVIDADE', 'CEP', 'UF', 'LOCALIDADE', 'BAIRRO', 'LOGRADOURO',
       'TELEFONE', 'FAX', 'E-MAIL2', 'E-MAIL3', 'SITE',
       'DATA GERAÇÃO CERTIFICADO', 'DATA VALIDADE CERTIFICADO',
       'CÓDIGO CERTIFICADO', 'CÓDIGO E DESCRIÇÃO CNAE', 'UH', 'UHs ACESSÍVEIS',
       'UHs PARA CAO GUIA', 'UHs TPS', 'TOTAL DE LEITOS', 'LINGUAS',
       'SEGMENTOS TURISTICOS', 'SERVIÇOS', 'EQUIPAMENTOS'],
      dtype='object')
Index(['RAZÃO SOCIAL', 'NOME FANTASIA', 'CNPJ', 'NATUREZA JURIDICA',
       'DATA INICIO OPERAÇÃO', 'PORTE', 'SITUAÇÃO', 'TIPO ATIVIDADE',
       'SUBTIPO ATIVIDADE', 'CEP', 'UF', 'LOCALIDADE', 'BAIRRO', 'LOGRADOURO',
       'TELEFONE', 'FAX', 'E-MAIL2', 'E-MAIL3', 'SITE',
       'DATA GERAÇÃO CERTIFICADO', 'DATA VALIDADE CERTIFICADO',
       'CÓDIGO CERTIFICADO', 'CÓDIGO E DESCRIÇÃO CNAE', 'UH', 'UHs ACESSÍVEIS',
       'UHs P

## Função Análise de Hospedagens
<p>Esta função <strong>análisa/filtra</strong> os dados oriundos dos CSV's,dentro da pasta <strong>Datasets</strong>, correspondentes dos <strong>meios de hospedagens</strong></p>

In [35]:
def AnaliseDadosHospedagens(dataFrame_Hospedagens):
    ##Escolhendo colunas
    dados = pd.DataFrame(dataFrame_Hospedagens,columns = ['UF', 'SITUAÇÃO', 'LINGUAS']) 

    ## Hospedagens que estão funcionando
    dados = dados[dados.SITUAÇÃO.str.contains('Em Operação')]

    ## Eliminando dados inválidos
    dados = dados.dropna()
    
    ## Pegando todos os idiomas oferecidos pelas hospedagens deixando de lado aqueles que não tem nenhum idioma
    allIdiomas = []
    for valor in dados.LINGUAS:
        valor_tokenize = word_tokenize(valor)
        for palavra in valor_tokenize:
            if (len(palavra) > 4) and (palavra != 'Nenhuma') and (palavra != 'Opções'):
                allIdiomas.append(palavra)

    linguas = []
    for idioma in allIdiomas:
        if idioma not in linguas:
            linguas.append(idioma)
    # Idiomas oferecidos pelas hospedagens
    print(linguas)

    # Adcionando Novas colunas
    for lingua in linguas:
        dados[lingua] = 0
    
    # Adicionando valores as coluncas de idiomas oferecidos
    for lingua in linguas:
        dados.loc[dados['LINGUAS'].str.contains(lingua),lingua] = 1
    
    return dados

## 1 Trimestre
<p>Análise dos <strong>dados</strong> das hospedagens nos meses de <strong>Janeiro</strong>,<strong>Fevereiro</strong> e <strong>Março</strong><p>

In [36]:
dadosHospedagens_Trimestre01 = AnaliseDadosHospedagens(hospedagens_trimestre01)
dadosHospedagens_Trimestre01.head()

['Francês', 'Inglês', 'Italiano', 'Espanhol', 'Alemão', 'Japonês', 'Hebraico', 'Russo', 'Mandarim']


,UF,SITUAÇÃO,LINGUAS,Francês,Inglês,Italiano,Espanhol,Alemão,Japonês,Hebraico,Russo,Mandarim
0,MA,Em Operação,"(""Francês,Inglês,Italiano"")",1,1,1,0,0,0,0,0,0
1,RS,Em Operação,"(""Nenhuma das Opções"")",0,0,0,0,0,0,0,0,0
2,SP,Em Operação,"(""Inglês,Espanhol"")",0,1,0,1,0,0,0,0,0
3,SP,Em Operação,"(""Inglês,Espanhol"")",0,1,0,1,0,0,0,0,0
4,SP,Em Operação,"(""Inglês,Espanhol"")",0,1,0,1,0,0,0,0,0


## 2 Trimestre
<p>Análise dos <strong>dados</strong> das hospedagens nos meses de <strong>Abril</strong>,<strong>Maio</strong> e <strong>Junho</strong><p>

In [37]:
dadosHospedagens_Trimestre02 = AnaliseDadosHospedagens(hospedagens_trimestre02)
dadosHospedagens_Trimestre02.head()

['Francês', 'Inglês', 'Italiano', 'Espanhol', 'Alemão', 'Japonês', 'Hebraico', 'Russo', 'Mandarim']


,UF,SITUAÇÃO,LINGUAS,Francês,Inglês,Italiano,Espanhol,Alemão,Japonês,Hebraico,Russo,Mandarim
0,MA,Em Operação,"(""Francês,Inglês,Italiano"")",1,1,1,0,0,0,0,0,0
1,RS,Em Operação,"(""Nenhuma das Opções"")",0,0,0,0,0,0,0,0,0
2,SP,Em Operação,"(""Inglês,Espanhol"")",0,1,0,1,0,0,0,0,0
3,SP,Em Operação,"(""Inglês,Espanhol"")",0,1,0,1,0,0,0,0,0
4,SP,Em Operação,"(""Inglês,Espanhol"")",0,1,0,1,0,0,0,0,0


## 3 Trimestre
<p>Análise dos <strong>dados</strong> das hospedagens nos meses de <strong>Julho</strong>,<strong>Agosto</strong> e <strong>Setembro</strong><p>

In [38]:
dadosHospedagens_Trimestre03 = AnaliseDadosHospedagens(hospedagens_trimestre03)
dadosHospedagens_Trimestre03.head()

['Italiano', 'Francês', 'Inglês', 'Espanhol', 'Alemão', 'Japonês', 'Hebraico', 'Russo', 'Mandarim', 'Eslovaco']


,UF,SITUAÇÃO,LINGUAS,Italiano,Francês,Inglês,Espanhol,Alemão,Japonês,Hebraico,Russo,Mandarim,Eslovaco
0,MA,Em Operação,"Italiano ,Francês ,Inglês",1,1,1,0,0,0,0,0,0,0
1,SP,Em Operação,"Espanhol ,Inglês",0,0,1,1,0,0,0,0,0,0
2,SP,Em Operação,"Espanhol ,Inglês",0,0,1,1,0,0,0,0,0,0
3,SP,Em Operação,"Espanhol ,Inglês",0,0,1,1,0,0,0,0,0,0
4,SP,Em Operação,Nenhuma das Opções,0,0,0,0,0,0,0,0,0,0


## 4 Trimestre
<p>Análise dos <strong>dados</strong> das hospedagens nos meses de <strong>Outubro</strong>,<strong>Novembro</strong> e <strong>Dezembro</strong><p>

In [39]:
dadosHospedagens_Trimestre04 = AnaliseDadosHospedagens(hospedagens_trimestre04)
dadosHospedagens_Trimestre04.head()

['Italiano', 'Francês', 'Inglês', 'Espanhol', 'Alemão', 'Japonês', 'Hebraico', 'Russo', 'Mandarim', 'Eslovaco', 'Catalão']


,UF,SITUAÇÃO,LINGUAS,Italiano,Francês,Inglês,Espanhol,Alemão,Japonês,Hebraico,Russo,Mandarim,Eslovaco,Catalão
0,MA,Em Operação,"Italiano ,Francês ,Inglês",1,1,1,0,0,0,0,0,0,0,0
1,SP,Em Operação,"Espanhol ,Inglês",0,0,1,1,0,0,0,0,0,0,0
2,SP,Em Operação,"Espanhol ,Inglês",0,0,1,1,0,0,0,0,0,0,0
3,SP,Em Operação,"Espanhol ,Inglês",0,0,1,1,0,0,0,0,0,0,0
4,SP,Em Operação,Nenhuma das Opções,0,0,0,0,0,0,0,0,0,0,0


In [51]:
dadosHospedagens_Trimestre04.describe

<bound method NDFrame.describe of       UF     SITUAÇÃO                              LINGUAS  Italiano  Francês  \
0     MA  Em Operação            Italiano ,Francês ,Inglês         1        1   
1     SP  Em Operação                     Espanhol ,Inglês         0        0   
2     SP  Em Operação                     Espanhol ,Inglês         0        0   
3     SP  Em Operação                     Espanhol ,Inglês         0        0   
4     SP  Em Operação                   Nenhuma das Opções         0        0   
5     RR  Em Operação                     Espanhol ,Inglês         0        0   
6     SP  Em Operação                     Espanhol ,Inglês         0        0   
7     TO  Em Operação                               Inglês         0        0   
8     PA  Em Operação                     Espanhol ,Inglês         0        0   
9     SP  Em Operação                     Espanhol ,Inglês         0        0   
10    MG  Em Operação                               Inglês         0       